# Exploratory Data Analysis (EDA) & Initial Cleaning

This notebook covers the initial exploration of the Loan Default dataset.
We will analyze:

1. Data Structure & Missing Values
2. Target Variable Distribution
3. Categorical Feature Analysis
4. Numerical Feature Distributions & Correlations
5. Key Insights for Modeling

## 1. Data Structure & Missing Values

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration for plots
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

In [ ]:
# Load the dataset
data_path = Path("data/raw/Loan_Default.csv")

if not data_path.exists():
    # Fallback if running from a different root
    data_path = Path("../data/raw/Loan_Default.csv")

df = pd.read_csv(data_path)
print(f"Data Loaded. Shape: {df.shape}")
df.head()

In [ ]:
df.info()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Percent': missing_percent})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Percent', ascending=False)

print("Features with Missing Values:")
display(missing_df)

In [ ]:
# Visualize missing values
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_df.index, y='Percent', data=missing_df, hue=missing_df.index, legend=False, palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title('Percentage of Missing Values per Feature')
plt.ylabel('Percentage (%)')
plt.tight_layout()
plt.show()

## 2. Target Variable Analysis (`Status`)

`Status` is our target variable:
*   0 = Non-Default
*   1 = Default

In [ ]:
target_counts = df['Status'].value_counts()
target_percent = df['Status'].value_counts(normalize=True) * 100

print("Target Distribution:")
print(target_counts)
print("\nPercentage:")
print(target_percent)

plt.figure(figsize=(6, 4))
sns.countplot(x='Status', data=df, hue='Status', legend=False, palette='pastel')
plt.title('Distribution of Loan Status (Target)')
plt.xticks([0, 1], ['Non-Default (0)', 'Default (1)'])
plt.show()

# 3. Categorical Feature Analysis

We investigate key categorical features and their relationship with the target.

In [ ]:
# Function to plot categorical feature vs Target
def plot_cat_vs_target(df, col, title=None, rotation=0):
    plt.figure(figsize=(10, 5))
    
    # Calculate default rate per category
    temp = df.groupby(col)['Status'].mean().sort_values(ascending=False) * 100
    
    sns.barplot(x=temp.index, y=temp.values, hue=temp.index, legend=False, palette='Reds_r')
    plt.title(title if title else f'Default Rate by {col}')
    plt.ylabel('Default Rate (%)')
    plt.xticks(rotation=rotation)
    plt.show()

### Credit Type Analysis

> While the dataset documentation does not explicitly define the values within the `credit_type` column, our external research identifies them as major credit reporting agencies. Preliminary analysis suggests this feature is strongly correlated with loan default outcomes, indicating that the source of the credit report may reflect the risk profile of the borrower. The definitions are as follows:
>
> * **EXP (Experian):** A multinational consumer credit reporting company headquartered in Dublin, Ireland. It is one of the 'Big Three' credit bureaus in the United States.
> * **EQUI (Equifax):** Another member of the 'Big Three' agencies, headquartered in Atlanta, Georgia, specializing in global consumer credit reporting.
> * **CRIF (Centrale Rischi Finanziari):** A global company specializing in credit bureau and business information systems, headquartered in Bologna, Italy, with a strong presence in Europe and Asia.
> * **CIB (Credit Information Bureau):** Likely refers to a generic Credit Information Bureau or a specific regional credit institution (e.g., Credit Information Bureau (India) Limited) used to assess borrower creditworthiness.

In [ ]:
plot_cat_vs_target(df, 'credit_type', title='Default Rate by Credit Type')

In [ ]:
# Verify the count of EQUI
print("Counts for Credit Type:")
print(df['credit_type'].value_counts())
print("\nDefault Rate for EQUI:")
equi_defaults = df[df['credit_type'] == 'EQUI']['Status'].mean()
print(f"{equi_defaults:.4%}")

### Age Analysis

In [ ]:
# Age is a string (e.g., "25-34"). Let's check the unique values and order them.
age_order = ['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74']
# Filter out any weird values if necessary, or just plot what we have
valid_ages = [x for x in age_order if x in df['age'].unique()]

plt.figure(figsize=(10, 5))
sns.countplot(x='age', data=df, order=valid_ages, hue='Status', palette='Set2')
plt.title('Loan Status by Age Group')
plt.show()

plot_cat_vs_target(df, 'age', title='Default Rate by Age Group')

### Region Analysis

In [ ]:
plot_cat_vs_target(df, 'Region', title='Default Rate by Region')

# 4. Numerical Feature Analysis

In [ ]:
# Select numerical columns
num_cols = df.select_dtypes(include=['number']).columns
# Drop ID and year (irrelevant)
num_cols = [c for c in num_cols if c not in ['ID', 'year']]

# Correlation Matrix
corr = df[num_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Heatmap (Numerical Features)')
plt.show()

print("Top Correlations with Status:")
print(corr['Status'].sort_values(ascending=False))

### Debt-to-Income Ratio (dtir1) vs Status

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Status', y='dtir1', data=df, hue='Status', palette='Set2')
plt.title('Debt-to-Income Ratio (dtir1) Distribution by Status')
plt.show()

In [ ]:
# Capping income for visualization because of outliers
q99 = df['income'].quantile(0.99)
temp_df = df[df['income'] < q99]

plt.figure(figsize=(10, 6))
sns.kdeplot(data=temp_df, x='income', hue='Status', fill=True, common_norm=False, palette='crest')
plt.title('Income Distribution by Status (Top 1% outliers removed)')
plt.show()